In [1]:
import pyfits
import numpy as np
import matplotlib as plt
from pylab import *
from astropy.cosmology import WMAP9 as cosmo
from matplotlib.colors import LogNorm

import scipy


import heapq
import pyfits
import matplotlib as plt
from pylab import *
import numpy as np
from decimal import *
import numpy
from scipy.optimize import curve_fit
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from pylab import *
from scipy import stats
from scipy.stats import chisquare
from astropy.io import fits
import copy



In [2]:
ins='Figs/kin_class_all_11.txt'
print(ins.split('.txt')[0].split('kin_class_')[1])
add_on=ins.split('.txt')[0].split('kin_class_')[1]


all_11


In [6]:
file_new = open('Figs/kin_table_'+str(add_on)+'.txt','w')
i=0
with open(ins, 'r') as f:
    data = f.readlines()
    for line in data:
        print(line)
        words[i] = line.split()
        
        print(words[i])
        
        if words[i][0]=='&':
            '''This is for reading two rows together'''
            '''First, determine how many Gaussian components is best'''
 
            if float(words[i][3].split('/')[0])/float(words[i][3].split('/')[1]) > 0.5 or float(words[i-1][4].split('/')[0])/float(words[i-1][4].split('/')[1]) > 0.5:
                num_gauss='$> 2$'
            else:
                num_gauss='2'
            #If its odd in length that means the divide is in the middle
            
            '''Next determine if velocity is above the threshhold value'''
            
            vel_before=float(words[i-1][6].split('\substack{+')[0].split('$')[1])
            if vel_before > 0:
                vel_before_plus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[0])
                vel_before_minus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            else:
                vel_before_minus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[0])
                vel_before_plus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            vel=float(words[i][5].split('\substack{+')[0].split('$')[1])
            if vel > 0:
                vel_plus=float(words[i][5].split('\substack{+')[1].split('\\\-')[0])
                vel_minus=float(words[i][5].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            else:
                vel_minus=float(words[i][5].split('\substack{+')[1].split('\\\-')[0])
                vel_plus=float(words[i][5].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
              
            if abs(vel) > 400 or abs(vel_before) > 400:
                '''Now were in the greater than or equal to regime'''
                
            
                if (abs(vel_before)-abs(vel_before_minus)) > 400 or (abs(vel) - vel_minus) > 400:
                    vel_cutoff='$> 400$'
                else:
                    vel_cutoff='$\geq 400$'
                    
            else:
                if abs(vel) < 400 and abs(vel_before) < 400:
                    if (abs(vel_before)+abs(vel_before_plus)) < 400 and (abs(vel)+abs(vel_plus)) < 400:
                        vel_cutoff='$< 400$'
                    else:
                        if (abs(vel_before)+abs(vel_before_plus)) > 400 or (abs(vel)+abs(vel_plus)) > 400:
                            vel_cutoff='$\leq 400$'
                        else:
                            vel_cutoff=':/'
                else:
                    vel_cutoff=':/'
            '''Disp overall now'''
            
            sig_overall=float(words[i][7].split('\substack{+')[0].split('$')[1])
            sig_plus=float(words[i][7].split('\substack{+')[1].split('\\\-')[0])
            sig_minus=float(words[i][7].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            
            sig_prev_overall=float(words[i-1][8].split('\substack{+')[0].split('$')[1])
            sig_prev_plus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[0])
            sig_prev_minus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            if sig_overall > 500 or sig_prev_overall > 500:
                if sig_overall - sig_minus < 500 and sig_prev_overall - sig_prev_minus < 500: 
                    sig_o='$\geq 500 $'
                else:
                    if sig_overall - sig_minus > 500 or sig_prev_overall - sig_prev_minus > 500:
                        sig_o='$> 500$'
                    else:
                        sig_o=':/'
            else:
                if sig_overall < 500 and sig_prev_overall < 500:
                    if sig_overall + sig_plus > 500 or sig_prev_overall + sig_prev_plus > 500:
                        sig_o='$\leq 500 $'
                    else:
                        if sig_overall + sig_plus < 500 and sig_prev_overall + sig_prev_plus < 500:
                            sig_o='$< 500$'
                        else:
                            sig_o=':/'
                else:
                    sig_o=':/'
                    
                    
            
            
            '''Doing the hardest part now with disperson of all four of the individual components'''
            
            #print(words[i][11].split('\substack{+')[0].split('$')[1])
            sig_1=float(words[i-1][8].split('\substack{+')[0].split('$')[1])
            sig_1_minus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_1_plus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[0])
            
            sig_2=float(words[i-1][10].split('\substack{+')[0].split('$')[1])
            sig_2_minus=float(words[i-1][10].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_2_plus=float(words[i-1][10].split('\substack{+')[1].split('\\\-')[0])
            
            sig_3=float(words[i][9].split('\substack{+')[0].split('$')[1])
            sig_3_minus=float(words[i][9].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_3_plus=float(words[i][9].split('\substack{+')[1].split('\\\-')[0])
            
            sig_4=float(words[i][11].split('\substack{+')[0].split('$')[1])
            sig_4_minus=float(words[i][11].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_4_plus=float(words[i][11].split('\substack{+')[1].split('\\\-')[0])
            
            
            
            if sig_1 > 500 or sig_2 > 500 or sig_3 > 500 or sig_4 > 500:
                '''This means were an outflow!'''
                if sig_1 - sig_1_minus > 500 or sig_2 - sig_2_minus > 500 or sig_3 - sig_3_minus > 500 or sig_4 - sig_4_minus > 500:
                    
                    sig_mult='$> 500$'
                else:
                    if sig_1 - sig_1_minus < 500 and sig_2 - sig_2_minus < 500 and sig_3 - sig_3_minus < 500 and sig_4 - sig_4_minus < 500:
                        sig_mult='$/geq 500$'
                    else:
                        sig_mult=':/'
            else:
                if sig_1 < 500 and sig_2 < 500 and sig_3 < 500 and sig_4 < 500:
                    if sig_1 + sig_1_plus > 500 or sig_2 + sig_2_plus > 500 or sig_3 + sig_3_plus > 500 or sig_4 + sig_4_plus > 500:
                        sig_mult='$/leq 500$'
                    else:
                        
                        if sig_1 + sig_1_plus < 500 and sig_2 + sig_2_plus < 500 and sig_3 + sig_3_plus < 500 and sig_4 + sig_4_plus < 500:
                            sig_mult='$< 500$'
                        else:
                            sig_mult=':/'
                else:
                    sig_mult=':/'
                    
            print(float(words[i-1][14]), float(words[i-1][16].split('$')[1]))
            PA_gal=float(words[i-1][14])
            PA_OIII=float(words[i-1][16].split('$')[1])
            
            if abs(PA_gal-PA_OIII) > 20:
                al='no'
            else:
                al='yes'
                
            '''Classification Time'''
            if sig_mult=='$> 500$' or sig_mult=='$/geq 500$':
                if num_gauss=='$> 2$':
                    classs='Outflow Composite'
                if num_gauss=='2':
                    classs='Outflow'
            else:
                if vel_cutoff=='$\geq 400$' or vel_cutoff=='$> 400$':
                    if num_gauss=='$> 2$':
                        classs='Outflow Composite'
                    if num_gauss=='2':
                        classs='Outflow'
                else:
                    if al=='no':
                        classs='Ambiguous'
                    if al=='yes':
                        classs='Rotation Dominated'
                
            file_new.write(str(words[i-1][0])+' & '+str(num_gauss)+' & '+str(vel_cutoff)+
                           ' & '+str(sig_o)+' & '+str(sig_mult)+' & '+str(al)+' & '+str(classs)+
                           '\\\ '+'\n')
        i+=1
        
file_new.close()
print('done')

J1604+5009 & 4 & 4/7 & $-139.8\substack{+3.8\\-7.2}$ & $142.2\substack{+29.9\\-19.3}$ & $352.3\substack{+173.9\\-60.8}$ & $183.9\substack{+0.3\\-42.8}$ & 120.1 & $98.2 \pm 5.4$ \\

['J1604+5009', '&', '4', '&', '4/7', '&', '$-139.8\\substack{+3.8\\\\-7.2}$', '&', '$142.2\\substack{+29.9\\\\-19.3}$', '&', '$352.3\\substack{+173.9\\\\-60.8}$', '&', '$183.9\\substack{+0.3\\\\-42.8}$', '&', '120.1', '&', '$98.2', '\\pm', '5.4$', '\\\\']
& 94 & 4/7 & $-495.7\substack{+6.8\\-4.0}$ & $334.8\substack{+29.6\\-30.6}$ & $132.6\substack{+30.9\\-0.0}$ & $313.6\substack{+294.6\\-103.1}$ &  &  \\

['&', '94', '&', '4/7', '&', '$-495.7\\substack{+6.8\\\\-4.0}$', '&', '$334.8\\substack{+29.6\\\\-30.6}$', '&', '$132.6\\substack{+30.9\\\\-0.0}$', '&', '$313.6\\substack{+294.6\\\\-103.1}$', '&', '&', '\\\\']
120.1 98.2
J1606+2736 & 70 & 4/11 & $-56.1\substack{+10.2\\-0.7}$ & $340.0\substack{+2.7\\-3.2}$ & $336.0\substack{+12.9\\-17.1}$ & $543.1\substack{+7.0\\-6.3}$ & 160.8 & $162.3 \pm 0.7$ \\

['J1606+2